In [2]:
# 필요한 패키지 설치 (없다면 실행)
# pip install langchain chromadb sentence-transformers

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [3]:

# 1) HuggingFace 임베딩 모델 지정 (DB 만든 모델과 동일해야 함)
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

# 2) 받은 DB 폴더 경로 (chroma.sqlite3, config.json 있는 위치)
persist_directory = "./duwls_chroma_db_sop"  

# 3) Chroma 로드
db = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

C:\Users\황여진\AppData\Local\Temp\ipykernel_30860\1564900351.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
c:\anaconda3\envs\final_pj\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [56]:
# 4) 쿼리 검색 테스트
query = "청소하기 시러" 
docs = db.similarity_search(query, k=3)

for i, doc in enumerate(docs, start=1):
    print(f"\n결과 {i}")
    print("내용:", doc.page_content[:300])   # 앞부분 300자 미리보기
    print("메타데이터:", doc.metadata)


In [57]:
# 전체 문서 수 확인
print("총 문서 수:", db._collection.count())

# 임의로 전체 문서 일부 꺼내보기
all_docs = db.get(include=["metadatas", "documents"], limit=3)
print(all_docs)


총 문서 수: 0
{'ids': [], 'embeddings': None, 'documents': [], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': []}


In [47]:
from chromadb import Client
from chromadb.config import Settings

client = Client(Settings(persist_directory="./rag data/chroma_db_gmp"))

# DB 안에 있는 컬렉션 이름 확인
collections = client.list_collections()
print(collections)


[]


In [ ]:
from chromadb import Client
from chromadb.config import Settings

client = Client(Settings(
    persist_directory="./rag data/chroma_db_gmp",  # 제작자 환경 경로 맞춤
    chroma_db_impl="duckdb+parquet"                # 제작자 백엔드와 동일하게
))

# 컬렉션 확인
print(client.list_collections())

# 컬렉션 로드
db = client.get_collection("컬렉션_이름")

print("총 문서 수:", db._collection.count())


# 23

In [24]:
# -*- coding: utf-8 -*-
"""
Pinecone GMP 벡터 DB RAG 시스템
저장된 벡터에서 유사 문서 검색 및 RAG 구현
"""

import os
from typing import List, Dict, Any, Optional
from pinecone import Pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from dotenv import load_dotenv


In [25]:
load_dotenv()

True

In [ ]:
class PineconeGMPRetriever:
    """Pinecone GMP 벡터 DB 검색기"""
    
    def __init__(self, 
                 index_name: str = "gmp-sop-vectordb",
                 embedding_model: str = "jhgan/ko-sroberta-multitask",
                 namespace: Optional[str] = None):
        """
        초기화
        
        Args:
            index_name: Pinecone 인덱스 이름
            embedding_model: 임베딩 모델 (저장 시와 동일해야 함)
            namespace: 검색할 네임스페이스 (None이면 기본)
        """
        self.index_name = index_name
        self.namespace = namespace
        
        # Pinecone 연결
        pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
        self.index = pc.Index(index_name)
        
        # 임베딩 모델 (저장 시와 동일한 모델 사용)
        self.embedder = HuggingFaceEmbeddings(
            model_name=embedding_model,
            encode_kwargs={'normalize_embeddings': True},
            model_kwargs={'device': 'cpu'}
        )
        
        print(f"✅ Pinecone 검색기 초기화 완료")
        print(f"   인덱스: {index_name}")
        print(f"   네임스페이스: {namespace or '기본'}")
        print(f"   임베딩 모델: {embedding_model}")
    
    def search(self, 
               query: str, 
               top_k: int = 5, 
               min_score: float = 0.7,
               filter_metadata: Optional[Dict] = None) -> List[Document]:
        """
        유사 문서 검색
        
        Args:
            query: 검색 쿼리
            top_k: 반환할 문서 수
            min_score: 최소 유사도 점수
            filter_metadata: 메타데이터 필터 (예: {"jurisdiction": "FDA"})
            
        Returns:
            List[Document]: 검색된 문서 리스트
        """
        # 쿼리 임베딩 생성
        query_vector = self.embedder.embed_query(query)
        
        # Pinecone 검색
        search_results = self.index.query(
            vector=query_vector,
            top_k=top_k,
            namespace=self.namespace,
            include_metadata=True,
            include_values=False,
            filter=filter_metadata  # 메타데이터 필터링
        )
        
        # Document 객체로 변환
        documents = []
        for match in search_results['matches']:
            # 최소 점수 필터링
            if match['score'] >= min_score:
                # 메타데이터에서 텍스트 내용 추출
                metadata = match.get('metadata', {})
                content = metadata.pop('text', '') if 'text' in metadata else ""
                
                # Document 생성
                doc = Document(
                    page_content=content,
                    metadata={
                        **metadata,
                        'score': match['score'],
                        'id': match['id']
                    }
                )
                documents.append(doc)
        
        print(f"🔍 검색 결과: {len(documents)}개 문서 (쿼리: '{query}')")
        return documents
    
    def search_by_namespace(self, 
                           query: str, 
                           namespaces: List[str], 
                           top_k: int = 3) -> Dict[str, List[Document]]:
        """
        여러 네임스페이스에서 동시 검색
        
        Args:
            query: 검색 쿼리
            namespaces: 검색할 네임스페이스 목록
            top_k: 네임스페이스별 반환할 문서 수
            
        Returns:
            Dict[str, List[Document]]: 네임스페이스별 검색 결과
        """
        query_vector = self.embedder.embed_query(query)
        results = {}
        
        for ns in namespaces:
            search_results = self.index.query(
                vector=query_vector,
                top_k=top_k,
                namespace=ns,
                include_metadata=True,
                include_values=False
            )
            
            documents = []
            for match in search_results['matches']:
                metadata = match.get('metadata', {})
                content = metadata.pop('text', '') if 'text' in metadata else ""
                
                doc = Document(
                    page_content=content,
                    metadata={
                        **metadata,
                        'score': match['score'],
                        'id': match['id'],
                        'namespace': ns
                    }
                )
                documents.append(doc)
            
            results[ns] = documents
        
        return results
    
    def get_stats(self) -> Dict:
        """인덱스 통계 조회"""
        return self.index.describe_index_stats()


In [39]:
rag1 = PineconeGMPRetriever(
    index_name="gmp-sop-vectordb",
    namespace="new-gmp-fda"
)

results = rag1.search(query="청소는 하기 싫다", top_k=5, min_score=0.1)

for doc in results:
    print(f"ID: {doc.metadata['id']}, 점수: {doc.metadata['score']}")
    print(f"내용: {doc.page_content}\n")

✅ Pinecone 검색기 초기화 완료
   인덱스: gmp-sop-vectordb
   네임스페이스: new-gmp-fda
   임베딩 모델: jhgan/ko-sroberta-multitask
🔍 검색 결과: 5개 문서 (쿼리: '청소는 하기 싫다')
ID: gti_lyophilization_of_parenterals-522ae25ad91c-0131, 점수: 0.388732
내용: 

ID: 6006a45874_gti_lyophilization_of_parenterals-e5e40c4ecc90-0131, 점수: 0.388732
내용: 

ID: 6006a45874_gti_lyophilization_of_parenterals-e5e40c4ecc90-0150, 점수: 0.36715892
내용: 

ID: gti_lyophilization_of_parenterals-522ae25ad91c-0150, 점수: 0.36715892
내용: 

ID: 0981942daf_Ch11Components-1bc68c034bac-0188, 점수: 0.363682717
내용: 



In [38]:
print(rag1.get_stats())


{'dimension': 768,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'fda-new-sop': {'vector_count': 1802},
                'new-gmp-fda': {'vector_count': 39281}},
 'total_vector_count': 41083,
 'vector_type': 'dense'}


In [51]:
results2 = rag1.search_by_namespace(query="청소는 하기 싫다", namespaces="new-gmp-fda", top_k=5)

In [54]:
for doc in results2:
    print(doc)

n
e
w
-
g
m
p
f
d
a
